In [1128]:
import os
from datetime import datetime

PROJECT_ID = "project_id"  # @param {type:"string"}
REGION = "asia-southeast1"

#### Bare-minimum framework to deploy endpoint on Vertex AI

In [1129]:
%%writefile requirements.txt
fastapi
uvicorn==0.17.6
joblib~=1.0
numpy~=1.20
scikit-learn~=1.3
google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform[prediction]>=1.16.0

Overwriting requirements.txt


In [1130]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [1131]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [1132]:
MODEL_ARTIFACT_DIR = "sklearn-cpr-preprocess-model-3"  # @param {type:"string"}
REPOSITORY = "custom-preprocess-container-prediction-3"  # @param {type:"string"}
IMAGE = "sklearn-cpr-preprocess-server-3"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "sklearn-cpr-preprocess-model-3"  # @param {type:"string"}

In [1133]:
BUCKET_NAME = "gcp_asia"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [1134]:
#! gsutil mb -l REGION -PROJECT_ID $BUCKET_URI

In [1135]:
#! gsutil ls -al $BUCKET_URI

In [1136]:
USER_SRC_DIR = "src_dir_3"  # @param {type:"string"}
LOCAL_MODEL_ARTIFACTS_DIR = "model_artifacts_3"  # @param {type:"string"}

In [1137]:
#%mkdir $USER_SRC_DIR $LOCAL_MODEL_ARTIFACTS_DIR

In [1138]:
#%%writefile $USER_SRC_DIR/preprocess.py
import numpy as np

class MySimpleScaler(object):
    def __init__(self):
        self._means = None
        self._stds = None

    def preprocess(self, data):
        if self._means is None:  # during training only
            self._means = np.mean(data, axis=0)

        if self._stds is None:  # during training only
            self._stds = np.std(data, axis=0)
            if not self._stds.all():
                raise ValueError("At least one column has standard deviation of 0.")

        return (data - self._means) / self._stds

In [1139]:
%cd $USER_SRC_DIR/

import pickle

import joblib
from preprocess import MySimpleScaler
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

iris = load_iris()
scaler = MySimpleScaler()

X = scaler.preprocess(iris.data)
y = iris.target

model = RandomForestClassifier()
model.fit(X, y)

#joblib.dump(model, f"../{LOCAL_MODEL_ARTIFACTS_DIR}/model.joblib")
#with open(f"../{LOCAL_MODEL_ARTIFACTS_DIR}/preprocessor.pkl", "wb") as f:
#    pickle.dump(scaler, f)

/home/jupyter/src_dir_3


RandomForestClassifier()

In [1140]:
%cd 
!gsutil cp {LOCAL_MODEL_ARTIFACTS_DIR}/* {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/
!gsutil ls {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

/home/jupyter
CommandException: No URLs matched: model_artifacts_3/*
CommandException: One or more URLs matched no objects.


In [1155]:
%%writefile $USER_SRC_DIR/predictor.py

import joblib
import numpy as np
import pickle

from google.cloud.aiplatform.prediction.sklearn.predictor import SklearnPredictor
from google.cloud.aiplatform.utils import prediction_utils
from google.cloud.aiplatform.prediction.handler import PredictionHandler
from google.cloud.aiplatform.prediction.handler import Handler
from google.cloud.aiplatform.prediction.predictor import Predictor
from google.cloud.aiplatform.prediction.serializer import DefaultSerializer

from sklearn.datasets import load_iris


class CprPredictor(Predictor):
    
    def __init__(self):
        return
    
    def load(self, artifacts_uri: str):
        """Loads the preprocessor and model artifacts."""
        #prediction_utils.download_model_artifacts(artifacts_uri)

        #with open("preprocessor.pkl", "rb") as f:
        #    preprocessor = pickle.load(f)

        self._class_names = load_iris().target_names
        #self._model = joblib.load("model.joblib")
        #self._preprocessor = preprocessor

    #def preprocess(self,instances):
    #    instances = instances["instances"]
    #    inputs = np.asarray(instances)
    #    preprocessed_inputs = self._preprocessor.preprocess(inputs)
        
    #    return inputs
    
    def predict(self, instances):
        """Performs prediction."""
        instances = instances["instances"]
        #inputs = np.asarray(instances)
        inputs=instances
        #preprocessed_inputs = self._preprocessor.preprocess(inputs)
        preprocessed_inputs=inputs
        #outputs = self._model.predict(preprocessed_inputs)

        return {"predictions": [[item[0],sum(item),np.mean(item)] for item in preprocessed_inputs]}

Overwriting src_dir_3/predictor.py


In [1156]:
!cp requirements.txt $USER_SRC_DIR/requirements.txt

In [1157]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [1158]:
import os

from google.cloud.aiplatform.prediction import LocalModel
from src_dir_3.predictor import CprPredictor
from google.cloud.aiplatform.prediction.handler import PredictionHandler
# Update this path as the variable $USER_SRC_DIR to import the custom predictor.

local_model = LocalModel.build_cpr_model(
    USER_SRC_DIR,
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    base_image= "python:3.10",
    predictor=CprPredictor,
    requirements_path=os.path.join(USER_SRC_DIR, "requirements.txt"),
)

/opt/conda/lib/python3.10/subprocess.py:955: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/opt/conda/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


In [1159]:
local_model.get_serving_container_spec()

image_uri: "asia-southeast1-docker.pkg.dev/teak-store-392915/custom-preprocess-container-prediction-3/sklearn-cpr-preprocess-server-3"
predict_route: "/predict"
health_route: "/health"

In [1160]:
INPUT_FILE = "instances.json"

In [1161]:
%%writefile $INPUT_FILE
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
}

Overwriting instances.json


In [1162]:
#!gcloud services list

In [1163]:
!gcloud artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

Create request issued for: [custom-preprocess-container-prediction-3]
Waiting for operation [projects/teak-store-392915/locations/asia-southeast1/ope
rations/4cf7c187-42ca-4a47-815a-d1462bd33604] to complete...done.              
Created repository [custom-preprocess-container-prediction-3].


In [1164]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "asia-southeast1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: asia-southeast1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [1165]:
local_model.push_image()

/opt/conda/lib/python3.10/subprocess.py:955: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/opt/conda/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


In [1166]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION)


In [1167]:
model = aiplatform.Model.upload(
    local_model=local_model,
    display_name=MODEL_DISPLAY_NAME,
    #artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
)


Creating Model
Create Model backing LRO: projects/958509004100/locations/asia-southeast1/models/8144381694099587072/operations/6397472122330939392
Model created. Resource name: projects/958509004100/locations/asia-southeast1/models/8144381694099587072@1
To use this Model in another session:
model = aiplatform.Model('projects/958509004100/locations/asia-southeast1/models/8144381694099587072@1')


In [1168]:
#endpoint = model.deploy(machine_type="n1-standard-2")